In [ ]:
using Dates
using JSON3
using DataFrames

In [ ]:
firstDate = Dates.DateTime(2023, 07, 19,   21, 0, 0)

In [ ]:
lastDate  = Dates.DateTime(2024, 01, 20,    9, 0, 0)

In [ ]:
measurementDate = collect(firstDate:Dates.Hour(6):lastDate);

In [ ]:
filenames = ["/home/lali/TITAN-ROG-sync/python/METEO/MET-Norway-"*i*".json" for i in string.(measurementDate)]
println("You have ", size(filenames)[1], " MET-Norway files   ->   ", last(filenames))

In [ ]:
function readOneFile(js)
    f = open(js,"r") 
    inDict = JSON3.read(f)
    close(f)
    
    return inDict
end 

In [ ]:
myDict = readOneFile(filenames[1])
@show nowTime = myDict.RO.now
@show nowTemp = myDict.RO.temp

In [ ]:
size(myDict.METNO)[1]

In [ ]:
#[convert(Dates.Hour, Dates.DateTime(myDict.METNO[D].now)  - Dates.DateTime(nowTime)) for D in 1:39]

In [ ]:
anticipateValues = [string(Dates.value(convert(Dates.Hour, Dates.DateTime(myDict.METNO[D].now)  - Dates.DateTime(nowTime) ))) for D in 1:size(myDict.METNO)[1]]

In [ ]:
allData = DataFrame([name => Float64[] for name in anticipateValues])

In [ ]:
insertcols!(allData, 1, :Data => DateTime[])
insertcols!(allData, 2, :Real => Float64[])
insertcols!(allData,    :"228" => Float64[]) # primul fișier are mai puține rânduri
allowmissing!(allData)

In [ ]:
#câte un rând pentru fiecare fișier
for measurement in measurementDate
    push!(allData, [measurement; missing; [missing for _ in anticipateValues]; missing])
end
#plus cele viitoare
for future in 1:size(myDict.METNO)[1]
    #println(lastDate + Hour(future*6))
    push!(allData, [lastDate + Hour(future*6); missing; [missing for _ in anticipateValues]; missing])
end

Empty DF ready!

In [ ]:
#filename = filenames[10]
for filename in filenames
    myDict = readOneFile(filename)
    nowTime = Dates.DateTime(myDict.RO.now)
    nowTemp = myDict.RO.temp
    #println("La data de: ", nowTime, "  a fost: ", nowTemp)
    myRow = Int(Dates.value(convert(Dates.Hour, nowTime  - firstDate))/6+1)
    allData[myRow,:Real] = nowTemp
    for i in myDict.METNO
        myRow = Int(Dates.value(convert(Dates.Hour, Dates.DateTime(i.now)  - firstDate))/6+1)
        myCol = string(Dates.value(convert(Dates.Hour, Dates.DateTime(i.now)  - nowTime)))
        #println("[", myRow, " ", myCol, "]: ", i.now, "  ", i.temp)
        allData[myRow,myCol] = i.temp
    end
end

DRAW

In [ ]:
x = [parse(Int, x) for x in anticipateValues]
y = allData[:,:Data]
z = fill(NaN, size(y, 1), size(x, 1))
for i in eachindex(x), j in eachindex(y)
    myRow = Int(Dates.value(convert(Dates.Hour, Dates.DateTime(y[j])  - firstDate))/6+1)
    if ismissing(allData[myRow, string(x[i])])
        z[j, i] = 0.0
    else
        z[j, i] = allData[myRow, string(x[i])]
    end
end

In [ ]:
using Plots

In [ ]:
heatmap(x, y, z)

In [ ]:
#using CairoMakie

In [ ]:
#f = Figure()
#Axis(f[1, 1])
#co = CairoMakie.contourf!(x, y, z)
#Colorbar(f[1, 2], co)
#f